# PRMT-2106: Produce 20 conversation IDs for EMIS top issues

## Context

Now that we have reproduced the new table with the re-categorisation of GP2GP transfer outcomes, we want to start prioritising the next issues to look at, and begin conversations with suppliers. We want to generate random samples of the top 2 ‘issues’ for each supplier, so that we can share these with them to review

## Scope 

Generate 20 conversation IDs in the last month for: 

- EMIS as a sender: 

    - Core extract not sent - Spine system responded with an error (err 20, sender?  Technical failure, core extract not sent)

    - Failed to successfully generate EHR extract (err 10, sender? Technical failure, contains fatal sender error)

- EMIS as a receiver: 

    - Large message re-assembly failure. (err 29, final? Technical failure, transferred not integrated with error)

- Vision as a sender:

    - Core extract not sent (Technical failure, core extract not sent)
- TPP to TPP

In [1]:
import pandas as pd

In [2]:
transfer_data_file = "s3://prm-gp2gp-data-sandbox-dev/transfers-sample-6/2021-6-transfers.parquet"
transfers = pd.read_parquet(transfer_data_file)

In [3]:
# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

# Generate ASID lookup that contains all the most recent entry for all ASIDs encountered
asid_file_location = "s3://prm-gp2gp-data-sandbox-dev/asid-lookup/asidLookup-Jun-2021.csv.gz"
asid_lookup = pd.read_csv(asid_file_location)
  
asid_lookup = asid_lookup.drop_duplicates().groupby("ASID").last().reset_index()
lookup = asid_lookup[["ASID", "MName", "NACS","OrgName"]]

transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'NACS': 'requesting_ods_code','OrgName':'requesting_practice_name'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'NACS': 'sending_ods_code','OrgName':'sending_practice_name'}, axis=1)

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

### EMIS as a sender

In [5]:
# Core extract not sent - Spine system responded with an error
emis_sender_bool = (transfers["sending_supplier"] == "EMIS")
spine_responded_with_error_bool = (transfers["sender_error_code"] == 20)
technical_failure_bool = (transfers["status"] == "TECHNICAL_FAILURE")
extract_not_sent_bool = (transfers["failure_reason"] == "Core Extract not Sent")
transfers_emis_error_20 = transfers[emis_sender_bool & spine_responded_with_error_bool & technical_failure_bool & extract_not_sent_bool]
transfers_emis_error_20[["conversation_id", "date_requested", "requesting_ods_code", "sending_ods_code", "status", "failure_reason", "sending_supplier", "sender_error_code"]].sample(20)

,conversation_id,date_requested,requesting_ods_code,sending_ods_code,status,failure_reason,sending_supplier,sender_error_code
94574,10D1B604-600A-4645-B9A4-BDE4CB5071D5,2021-06-03 09:27:51.477,N83021,N83020,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
220281,4C7F9E00-CF74-11EB-BE51-89E37BF56130,2021-06-17 13:59:57.495,H82020,G82015,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
185554,4D67A506-6568-4058-98B8-3D99AEF87F9D,2021-06-24 09:00:29.442,F82013,F84031,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
13344,DFAF9F54-3679-4FCC-9549-F46A7DB31D84,2021-06-11 11:00:26.237,M84037,M86020,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
58140,515B70D6-9904-49BA-8EEB-C9DD5DCB2D36,2021-06-09 09:28:58.655,J82639,J82035,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
247234,B6C93C4E-AEE5-4B65-9118-A36B7D607F7E,2021-06-14 14:00:09.903,K81025,K81022,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
160713,AA7B5A80-D438-11EB-A20E-EB86DB7EE1A0,2021-06-23 15:35:41.165,E82079,F85678,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
95019,A78312EE-5158-4734-9642-1492D3F968C3,2021-06-03 10:23:58.934,H85048,Y02946,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
32599,13B1FE60-C2EA-11EB-90BB-4748DB89A434,2021-06-01 15:00:17.794,C82021,N82108,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0
134658,0FDAEC2A-97F1-4F4B-B2ED-93011068FC0C,2021-06-29 11:30:34.945,M84030,C84117,TECHNICAL_FAILURE,Core Extract not Sent,EMIS,20.0


In [6]:
# Failed to successfully generate EHR extract
error_code_10_bool = (transfers["sender_error_code"] == 10)
transfers_emis_error_10 = transfers[emis_sender_bool & error_code_10_bool & technical_failure_bool]
transfers_emis_error_10[["conversation_id", "date_requested", "requesting_ods_code", "sending_ods_code", "status", "failure_reason", "sending_supplier", "sender_error_code"]].sample(20)

,conversation_id,date_requested,requesting_ods_code,sending_ods_code,status,failure_reason,sending_supplier,sender_error_code
89603,44BEA261-2610-493C-A0F7-9334F4975C50,2021-06-01 14:26:49.134,F85063,G85034,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
244456,9AB83560-CDCF-11EB-8165-8114FBDA43C3,2021-06-15 11:48:31.074,L81071,G85726,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
188725,C62432B5-EEA1-425E-B7CC-B0DE748405FB,2021-06-23 11:23:06.827,F84035,G85132,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
99755,BFC60C7A-4627-4791-A4C6-5A8048B02F0B,2021-06-02 10:35:37.097,H85007,G85685,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
217638,6851710D-EF81-46A8-B1CD-0637F1480E4C,2021-06-21 08:15:27.845,M85019,H85649,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
91037,D6CE2847-A4AD-484B-90FF-F9DECEC9CB70,2021-06-03 12:47:49.042,K84002,G85042,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
87208,886C8597-B45B-4888-8103-1B36D7D88EA2,2021-06-01 11:46:02.315,F83683,F83635,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
9457,FC85F3A0-CA9F-11EB-A03E-CBE901DD621B,2021-06-11 10:30:05.387,M89005,G85132,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
244514,856DB820-CDBF-11EB-86E5-093F1D2BA9D0,2021-06-15 09:53:22.959,E85055,G85132,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0
45799,FB1B6375-CEA4-4FC8-9BD2-3A2A98B8ABD2,2021-06-09 15:07:45.430,H83004,G85685,TECHNICAL_FAILURE,Contains Fatal Sender Error,EMIS,10.0


### EMIS as a receiver

In [7]:
#  check if 29 exists in final error codes
transfers_split_by_final_error_code=transfers.explode("final_error_codes")
transfers_split_by_final_error_code["final_error_codes"].value_counts()

15.0    16906
12.0     4841
31.0      703
11.0      671
99.0      351
25.0      332
30.0      127
28.0      127
17.0      108
20.0       20
21.0       17
9.0         1
Name: final_error_codes, dtype: int64

In [8]:
#  check if 29 exists in intermediate error codes
transfers_split_by_intermediate_error_code=transfers.explode("intermediate_error_codes")
transfers_split_by_intermediate_error_code["intermediate_error_codes"].value_counts()

29    972
Name: intermediate_error_codes, dtype: int64

In [13]:
# Large message re-assembly failure
emis_receiver_bool = (transfers_split_by_intermediate_error_code["requesting_supplier"] == "EMIS")
error_code_29_bool = (transfers_split_by_intermediate_error_code["intermediate_error_codes"] == 29)
transfers_emis_error_29 = transfers_split_by_intermediate_error_code[emis_receiver_bool & error_code_29_bool & technical_failure_bool]
transfers_emis_error_29[["conversation_id", "date_requested", "requesting_ods_code", "sending_ods_code", "status", "failure_reason", "sending_supplier", "intermediate_error_codes", "final_error_codes"]].sample(20)

,conversation_id,date_requested,requesting_ods_code,sending_ods_code,status,failure_reason,sending_supplier,intermediate_error_codes,final_error_codes
125287,D484C13F-4431-4D8B-A9B3-1D2C023D1B7C,2021-06-28 13:58:14.079,C81086,C88015,TECHNICAL_FAILURE,Final Error,SystmOne,29,[31.0]
159577,5398E6F1-41ED-4718-8D3B-31165F1C957E,2021-06-17 09:43:14.848,N83601,N83055,TECHNICAL_FAILURE,Final Error,EMIS,29,"[31.0, 31.0, 12.0]"
17088,87A91F12-5818-475C-8D82-C9BD441EE852,2021-06-09 15:58:28.586,P81006,Y02322,TECHNICAL_FAILURE,Final Error,EMIS,29,[31.0]
7092,266970D8-897A-4544-AE32-5183368EB3E1,2021-06-11 09:56:39.542,G82107,K83621,TECHNICAL_FAILURE,Final Error,SystmOne,29,[25.0]
74986,F306C10C-7315-4082-B9AE-DB746F0C8676,2021-06-02 14:51:50.789,L81022,L81023,TECHNICAL_FAILURE,Final Error,EMIS,29,[31.0]
59463,04A60206-C612-40FE-A1D9-7BDBAD5B3EDE,2021-06-02 13:00:13.326,L81118,D81607,TECHNICAL_FAILURE,Final Error,SystmOne,29,[25.0]
232041,B6A9E768-FFFF-4D19-8A20-82F0FA702D5E,2021-06-14 12:19:10.513,F82678,F86702,TECHNICAL_FAILURE,Final Error,EMIS,29,[31.0]
129672,BDB516C7-A304-4F4F-B566-209E0ED7A26C,2021-06-29 13:45:51.000,N81607,N81030,TECHNICAL_FAILURE,Final Error,EMIS,29,[31.0]
166780,71E5F16F-9422-449F-B436-40D9FFAB6D8B,2021-06-23 10:54:09.055,J82012,J82155,TECHNICAL_FAILURE,Final Error,SystmOne,29,[31.0]
149705,817241B0-F0BB-46AC-9520-B1FD18C5DE58,2021-06-21 11:59:40.294,B81029,C88007,TECHNICAL_FAILURE,Final Error,SystmOne,29,[31.0]


### Vision as a sender

In [30]:
# Core extract not sent 
vision_sender_bool = (transfers["sending_supplier"] == "Vision")
transfers_vision_extract_not_sent = transfers[vision_sender_bool & technical_failure_bool & extract_not_sent_bool]
transfers_vision_extract_not_sent[["conversation_id", "date_requested", "requesting_ods_code", "sending_ods_code", "status", "failure_reason", "sending_supplier"]].sample(20)

,conversation_id,date_requested,requesting_ods_code,sending_ods_code,status,failure_reason,sending_supplier
234036,7D4EAF78-FF4D-4142-9810-C1A8BEF03EF2,2021-06-17 09:58:46.056,P83001,P83004,TECHNICAL_FAILURE,Core Extract not Sent,Vision
230425,0BC3FE94-D032-4774-8282-FFAE6EEF77FB,2021-06-17 10:02:37.417,P83001,P83609,TECHNICAL_FAILURE,Core Extract not Sent,Vision
95792,909615A0-C3A4-11EB-BA1C-3FFFD7755875,2021-06-02 13:15:13.521,J84011,G83027,TECHNICAL_FAILURE,Core Extract not Sent,Vision
234005,305D635F-296B-45F1-B82F-4A2223077DA2,2021-06-17 10:57:09.792,P83623,P83605,TECHNICAL_FAILURE,Core Extract not Sent,Vision
233677,B9C32020-CF53-11EB-B1A5-431A81FC1CD2,2021-06-17 10:06:47.451,L81617,H85027,TECHNICAL_FAILURE,Core Extract not Sent,Vision
73919,467BBD40-C7B3-11EB-BD15-839DE9E26332,2021-06-07 17:10:36.329,P82008,P83021,TECHNICAL_FAILURE,Core Extract not Sent,Vision
209349,91157975-6E84-469F-BB57-448D3A1CE84B,2021-06-22 08:50:23.990,L83016,H84002,TECHNICAL_FAILURE,Core Extract not Sent,Vision
133696,B5467E71-DCD2-4ABB-BE20-25068DC11457,2021-06-29 10:11:56.689,P81757,P87620,TECHNICAL_FAILURE,Core Extract not Sent,Vision
162279,44FB95E1-9B48-4E32-9987-81188A720565,2021-06-25 14:26:45.971,G85055,G83058,TECHNICAL_FAILURE,Core Extract not Sent,Vision
202321,27673AE2-BF2B-4DE1-9C0E-48DBE211FA98,2021-06-17 11:41:53.016,P83623,P83001,TECHNICAL_FAILURE,Core Extract not Sent,Vision


### TPP to TPP

In [27]:
tpp_sender_bool = (transfers["sending_supplier"] == "SystmOne")
tpp_requestor_bool = (transfers["requesting_supplier"] == "SystmOne")

tpp_to_tpp_transfers = transfers[tpp_sender_bool & tpp_requestor_bool]
tpp_to_tpp_transfers[["conversation_id", "date_requested", "requesting_ods_code", "sending_ods_code", "status", "failure_reason", "sending_supplier", "requesting_supplier"]].sample(11)

,conversation_id,date_requested,requesting_ods_code,sending_ods_code,status,failure_reason,sending_supplier,requesting_supplier
47443,9A304BA0-C938-11EB-8A40-6FDC502F3E34,2021-06-09 15:37:31.088,C87604,NaN,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
29754,B4298CF0-CA92-11EB-BA97-B7D2D2F1F99D,2021-06-11 08:55:00.436,C88039,NaN,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
103792,A73CEC90-D9C3-11EB-9AC4-8D3899767012,2021-06-30 16:53:12.211,Y00561,E81611,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
95111,1DD8E200-C3B2-11EB-A9BA-F5ABFDCE7ECB,2021-06-02 14:52:14.314,M89003,NaN,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
117925,E4AC9B40-D98F-11EB-B1A3-E99FAB9C8841,2021-06-30 10:42:41.358,Y00560,E81611,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
104039,493089E0-D9C3-11EB-9AC4-8D3899767012,2021-06-30 16:50:34.421,Y00561,E81611,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
102784,E6FDAF00-D9C2-11EB-9AC4-8D3899767012,2021-06-30 16:47:49.494,Y00561,E81611,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
27592,1AE201E0-CA91-11EB-BA97-B7D2D2F1F99D,2021-06-11 08:43:34.144,C88039,NaN,TECHNICAL_FAILURE,Contains Fatal Sender Error,SystmOne,SystmOne
59465,2EFC1230-C8F0-11EB-BB71-C79CFBE47C50,2021-06-09 06:59:07.703,E87046,E87017,TECHNICAL_FAILURE,Request not Acknowledged,SystmOne,SystmOne
153704,AA45D5D0-D59C-11EB-9185-990BFFBB2A3C,2021-06-25 10:04:02.166,E87714,E87017,TECHNICAL_FAILURE,Request not Acknowledged,SystmOne,SystmOne
